In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy as sp
import torch
from torch.autograd import Variable
from torch import nn
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

In [37]:
raw_data=pd.read_csv('meta_data/Data1.csv')
label_1=pd.read_csv('meta_data/Label_of_Data1.csv')
label_1.iloc[86,2]=30014

length=label_1.iloc[:,2]-label_1.iloc[:,1]
length=pd.DataFrame(length,columns=['length']).to_numpy()

label_1=label_1.to_numpy()
label_1=np.append(label_1,length,axis=1)
label_1[:,0]=np.where(label_1[:,0]>'-1',1,0)
respond_label_1=label_1[:,0]
training_set_label=label_1[:,-1].reshape(126,1)

data_set=raw_data.iloc[:,1:7].to_numpy()
response_transition=raw_data.iloc[:46000,-2:-1].to_numpy()
response_gesture=raw_data.iloc[:46000,-1:].to_numpy()
Time=raw_data.iloc[:46000,0]
E_acc=(raw_data.iloc[:,1]**2 + raw_data.iloc[:,2]**2 + raw_data.iloc[:,3]**2).to_numpy().reshape(len(data_set),1)
E_omega=(raw_data.iloc[:,4]**2 + raw_data.iloc[:,5]**2 + raw_data.iloc[:,6]**2).to_numpy().reshape(len(data_set),1)
data_set=np.append(data_set,E_acc,axis=1)
data_set=np.append(data_set,E_omega,axis=1)

In [38]:

#   进行标准化

# scaler=StandardScaler()

# data_train1=scaler.fit_transform(data_set)  #   以每个点来做分类

# training_set_label=scaler.fit_transform(training_set_label)
data_train1=data_set[:46000,:]




loss_func=F.multilabel_soft_margin_loss
data_train1=torch.from_numpy(data_train1)
data_train1=data_train1.float()
data_train1=Variable(data_train1,requires_grad=True)
y_trs=torch.from_numpy(response_transition)
y_trs.type_as(data_train1)



tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [39]:
D_in,H1,H2,H3,H4,D_out=8,8,8,8,4,2

class gesture_NN(nn.Module):
    def __init__(self,D_in,H1,H2,H3,H4,D_out):
        super(gesture_NN,self).__init__()
        self.hidden1=nn.Linear(D_in,H1)
        self.hidden2=nn.Linear(H1,H2)
        self.hidden3=nn.Linear(H2,H3)
        self.hidden4=nn.Linear(H3,H4)
        self.out=nn.Linear(H4,D_out)
        

        
    def forward(self,x):
        x=F.tanh(self.hidden1(x))
        x=F.sigmoid(self.hidden2(x))
        x=F.relu(self.hidden3(x))
        x=F.relu(self.hidden4(x))
        x=self.out(x)
        x=F.softmax(x)
        return x

net=gesture_NN(D_in,H1,H2,H3,H4,D_out)


In [40]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

X_train=data_train1
Y_train=y_trs

train_ds=TensorDataset(X_train,Y_train)
train_dl=DataLoader(train_ds,batch_size=5000,shuffle=True)



In [41]:
def get_model(net):
    model=net
    return model,optim.SGD(model.parameters(),lr=1e-4)


def loss_batch(model,loss_func,xb,yb,opt=None):
    loss=loss_func(model(xb),yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(),len(xb)



def fit(steps,model,loss_func,opt,train_dl):
    for step in range(steps):
        model.train()
        for xb,yb in train_dl:
            loss,len=loss_batch(model,loss_func,xb,yb,opt)
            print(loss)

In [42]:
# model,opt=get_model(net)
# fit(5000,model,loss_func,opt,train_dl)


In [43]:
#   对一段时间序列进行RNN
seq_len=100    
hidden_size=8
batch_size=1
feature_len=8   #输入特征数
num_layers=4
outputsize=2
x=X_train.view(batch_size,-1,feature_len)
y=Y_train.view(batch_size,-1,1)
train_ds=TensorDataset(x,y)
train_dl=DataLoader(train_ds,batch_size=1,shuffle=True)

In [46]:

class net_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn=rnn=nn.RNN(input_size=feature_len,hidden_size=hidden_size,num_layers=4,batch_first=True)
        self.linear=nn.Linear(hidden_size,outputsize)
        self.softmax=nn.Softmax(dim=0)
    def forward(self,x,hidden_pre):
        out,hidden_pre=self.rnn(x,hidden_pre)
        out=out.view(-1,hidden_size)
        out=self.linear(out)
        out=out.unsqueeze(dim=0)
        out=self.softmax(out)
        return out,hidden_pre


net=net_rnn()

model,opt=get_model(net)
hidden_pre=torch.zeros(num_layers,batch_size,hidden_size)

for iter in range(50000):
    model.train()
    for xb,yb in train_dl:
        output,hidden_pre=model(xb,hidden_pre)
        hidden_pre=hidden_pre.detach()

        loss=loss_func(output,yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    if iter%10==0:
        print(loss)

tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)
tensor(1.1042, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 

In [48]:


path='model/RNN_model.pt'
torch.save(model,path)


In [238]:
# class lstm(nn.Module):
#     def __init__(self, feature_len, hidden_size, num_layers, num_classes):
#         super(lstm, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(feature_len, hidden_size, num_layers, 
#                             batch_first=True)
#         self.fc = nn.Linear(hidden_size, num_classes)  # 2 for bidirection 

#     def forward(self, x):

#         # Forward propagate RNN
#         out, _ = self.lstm(x)

#         # Decode hidden state of last time step
#         out = self.fc(out[:, -1, :])
#         return out

In [243]:
# net_lstm=lstm(8,8,3,1)
# model_lstm,opt_lstm=get_model(net_lstm)
# hidden_pre=torch.zeros(num_layers,batch_size,hidden_size)
# for iter in range(500):
#     model.train()
#     for xb,yb in train_dl:
#         output,hidden_pre=model_lstm(xb,hidden_pre)
#         hidden_pre=hidden_pre.detach()

#         loss=loss_func(output,yb)
#         loss.backward()
#         opt.step()
#         opt.zero_grad()
        
#         print(loss)

TypeError: forward() takes 2 positional arguments but 3 were given

In [256]:
# for name,parameter in model.named_parameters():
#     print(parameter)


raw_data2=pd.read_csv('meta_data/Data2.csv')
data_set2=raw_data2.iloc[:,1:7].to_numpy()

Time=raw_data2.iloc[:,0]
E_acc2=(raw_data2.iloc[:,1]**2 + raw_data2.iloc[:,2]**2 + raw_data2.iloc[:,3]**2).to_numpy().reshape(len(data_set2),1)
E_omega2=(raw_data2.iloc[:,4]**2 + raw_data2.iloc[:,5]**2 + raw_data2.iloc[:,6]**2).to_numpy().reshape(len(data_set2),1)
data_set2=np.append(data_set2,E_acc2,axis=1)
data_set2=np.append(data_set2,E_omega2,axis=1)

data_set2=torch.from_numpy(data_set2)
data_set2=data_set2.float()

x_pre=data_set2.view(batch_size,-1,feature_len)

# train_ds_pre=TensorDataset(x_pre)
# train_dl_pre=DataLoader(train_ds,batch_size=5000,shuffle=True)
pre=[]


model.eval()
for i in range(x_pre.shape[1]):
    input=x_pre[:,i,:]
    input=input.view(1,1,8)
    pred,hidden_pre=model(input,hidden_pre)
    input=pred
    pre.append(pred.detach().numpy().ravel())




[array([-0.03726299, -0.02404223], dtype=float32), array([-0.03861575, -0.01913722], dtype=float32), array([-0.03318109, -0.02114588], dtype=float32), array([-0.03770693, -0.0206129 ], dtype=float32), array([-0.03421231, -0.02136472], dtype=float32), array([-0.03928582, -0.01891173], dtype=float32), array([-0.0328874 , -0.02141321], dtype=float32), array([-0.03777422, -0.02085827], dtype=float32), array([-0.03747572, -0.01807207], dtype=float32), array([-0.03358082, -0.02091764], dtype=float32), array([-0.03799579, -0.01997146], dtype=float32), array([-0.03628333, -0.0185668 ], dtype=float32), array([-0.03241557, -0.02389574], dtype=float32), array([-0.03963873, -0.02017836], dtype=float32), array([-0.03636235, -0.01663882], dtype=float32), array([-0.03825387, -0.01685727], dtype=float32), array([-0.03714684, -0.01735389], dtype=float32), array([-0.03414662, -0.02123037], dtype=float32), array([-0.03656761, -0.02146195], dtype=float32), array([-0.03773795, -0.01924434], dtype=float32),